In [ ]:
import warnings,os,shutil
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from pymatgen.core import Lattice, Structure
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from m3gnet.models import Relaxer, M3GNet
from pymatgen.core import Lattice, Structure

relaxer = Relaxer()  

def distance_list(min_distance,target_indices,ga_length_total):
    distant_indice_list = []
    temp_list = []
    for i in range(0,ga_length_total):
        if i > max(target_indices):
            temp_list += [i]
    for i in temp_list:
        min_distance_list = []
        for target_index in target_indices:
            distance = structure.get_distance(target_index, i)
            min_distance_list += [distance]
        if min(min_distance_list) > min_distance:
            distant_indice_list += [i]
    return distant_indice_list

def test_are_symmetrically_equivalent(structure,indice_lists,test_indice):
    for indice in indice_lists:
        sites1 = [structure[i] for i in indice]
        sites2 = [structure[i] for i in test_indice]
        sg1 = SpacegroupAnalyzer(structure, 0.01).get_space_group_operations()
        if sg1.are_symmetrically_equivalent(sites1, sites2, 1e-1) == True:
            return True
            break
    return False

def generate_structures(min_distance, num_limit, structure, folder_path, ga_length_remove, ga_length_total):
    indice_list = []
    num = 0
    ga_indices = [0] * ga_length_remove

    def recursive_structure(index):
        nonlocal num
        if index == ga_length_remove:  # If the index reaches the length of ga_indices
            if num > num_limit:
                return
            if len(indice_list) == 0:
                new_list = ga_indices.copy()
                indice_list.append(new_list)
            else:
                if test_are_symmetrically_equivalent(structure, indice_list, ga_indices):
                    return
                else:
                    new_list = ga_indices.copy()
                    indice_list.append(new_list)
            possible_structure = structure.copy()
            possible_structure.remove_sites(ga_indices)
            filename = f"{folder_path}/{ga_indices}.vasp"
            poscar = Poscar(possible_structure)
            poscar.write_file(filename)
            num += 1
            return

        # Set initial value for ga_indices at the current index
        for i in [0, 4] if index == 0 else distance_list(min_distance, ga_indices, ga_length_total):
            if num > num_limit:
                break
            ga_indices[index] = i
            if index < ga_length_remove - 1 and len(distance_list(min_distance, ga_indices, ga_length_total)) == 0:
                continue
            recursive_structure(index + 1)
            ga_indices[index] = ga_indices[0]

    recursive_structure(0)
    
folder_path = "1_221_wo_VGa"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
if not os.path.exists(folder_path+'_relaxed'):
    os.makedirs(folder_path+'_relaxed')
structure = Structure.from_file("SPOSCAR_221_spinel.vasp")

min_distance = 5
num_limit = 50
ga_length_remove = 4
ga_length_total = 32

generate_structures(min_distance,num_limit,structure,folder_path,ga_length_remove,ga_length_total)
file_names = os.listdir(folder_path)
for name in file_names:
    struct = Structure.from_file(f"{folder_path}/{name}")
    relax_results = relaxer.relax(struct, fmax=0.01)
    # extract results
    final_structure = relax_results["final_structure"]
    final_structure.to(f"{folder_path}_relaxed/relaxed_{name}",fmt="poscar")
    final_energy = relax_results["trajectory"].energies[-1]
    with open("output.txt", "a") as f:
        print(f"{name}, {float(final_energy):.3f}")